In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [16]:
import numpy as np

### catom metric

In [89]:
class P4Metric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        # get probs and preds
        approx = approxes[0]
        e = np.exp(approx)
        p = e / (1 + e)
        preds = (p > 0.5).astype(int)
        target = np.array(target).astype(int)
            
        tn = ((preds == 0) & (target == 0)).sum()
        fn = ((preds == 0) & (target == 1)).sum()
        fp = ((preds == 1) & (target == 0)).sum()
        tp = ((preds == 1) & (target == 1)).sum()

        p4 = (4 * tp * tn) / (4* tp * tn + (tp + tn) * (fp + fn))
        return p4, 1

### check

In [10]:
path = 'https://stepik.org/media/attachments/lesson/779917/CB_bin_target_data.csv'
rides_info = pd.read_csv(path)
cat_cols = ['car_type', 'fuel_type', 'model']
X = rides_info.drop(['target_bin'], axis=1)
y = rides_info['target_bin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [83]:
model = CatBoostClassifier(random_state=42,
                           cat_features=cat_cols,
                           eval_metric=P4Metric())

model.fit(X_train, y_train, 
          eval_set=(X_test, y_test),
          verbose=50, plot=False, 
          early_stopping_rounds=100)

Learning rate set to 0.036983
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 79.7ms	remaining: 1m 19s


/home/katsiaryna/.conda/envs/ranking/lib/python3.9/site-packages/catboost/core.py:1761: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of BoundFunction(array.astype for array(bool, 1d, C)) with parameters (Function(<class 'int'>))

During: resolving callee type: BoundFunction(array.astype for array(bool, 1d, C))
During: typing of call at /tmp/ipykernel_36924/4218925392.py (15)


File "../../../../../tmp/ipykernel_36924/4218925392.py", line 15:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


50:	learn: 0.6618212	test: 0.5148397	best: 0.5148397 (50)	total: 482ms	remaining: 8.96s
100:	learn: 0.8116578	test: 0.5942377	best: 0.5989402 (92)	total: 891ms	remaining: 7.93s
150:	learn: 0.8649428	test: 0.6610884	best: 0.6610884 (145)	total: 1.32s	remaining: 7.42s
200:	learn: 0.8952646	test: 0.6438122	best: 0.6779758 (178)	total: 1.71s	remaining: 6.8s
250:	learn: 0.9362061	test: 0.6305286	best: 0.6779758 (178)	total: 2.12s	remaining: 6.31s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6779758482
bestIteration = 178

Shrink model to first 179 iterations.
